<a href="https://colab.research.google.com/github/jasog01/cursomlean/blob/main/Eevaluaci%C3%B3n_de_calidad_de_carros_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Proyecto 2 – Modelos de clasificación supervisada para evaluar la calidad de un automovil

**Integrantes del Grupo:**
- Jose Sogamoso
- Andres Marin
- Blexy Granados

**Descripción del Problema:**

Analisis de evaluación de automoviles considerando seis caracteristicas: precio, mantenimiento, puertas, personas, maletero, seguridad.  Basado en esto categorizamos los vehiculos en las siguientes clases:
unacc, acc, good, vgood.

Fuente de datos https://archive.ics.uci.edu/dataset/19/car+evaluation



#Resultados De comparar 3 modelos

##Comparativo de modelos

* Knn test accuracy: 0.96
* Random Forest accuracy :0.96
* LogisticRegression test accuracy: 0.93

Los modelos que mejor se ajustan a la clasificación del ejercicio son el KNeighborsClassifier y Random Fores, si bien la regresión logística tiene una precisión ligeramente más baja, en este caso particular, puedieramos inferir que la diferencia en el rendimiento entre la regresión logística y los otros modelos podría atribuirse, a la sensibilidad de la regresión logística a la escala de las características y tambien al balance en las etiquetas objetivo, ya que la regresión puede sesgarse hacia la clase mayoritaria.

In [153]:
pip install ucimlrepo

In [154]:
from ucimlrepo import fetch_ucirepo
  # fetch dataset
car_evaluation = fetch_ucirepo(id=19)

import pandas as pd
datos=car_evaluation['data']
df = pd.DataFrame(datos['original'])



#Analisis exploratorio
El dataset proporcionado cuenta con variables categoricas las cuales trasnformaremos a codificacion Ordinal, no require tratamiento de datos nulos.

In [155]:
car_evaluation['variables']

,name,role,type,demographic,description,units,missing_values
0,buying,Feature,Categorical,None,buying price,None,no
1,maint,Feature,Categorical,None,price of the maintenance,None,no
2,doors,Feature,Categorical,None,number of doors,None,no
3,persons,Feature,Categorical,None,capacity in terms of persons to carry,None,no
4,lug_boot,Feature,Categorical,None,the size of luggage boot,None,no
5,safety,Feature,Categorical,None,estimated safety of the car,None,no
6,class,Target,Categorical,None,"evaulation level (unacceptable, acceptable, go...",None,no


#Asignación de Codificación Variables categoricas

##Tecnica Seleccionada : Codificación Ordinal

In [156]:
from sklearn.preprocessing import OrdinalEncoder

#Codificacion Ordinal varaibles categoricas

ordinal_encoder = OrdinalEncoder()

# Ajustar y transformar la columna categórica
df['buying'] = ordinal_encoder.fit_transform(df[['buying']])
df['maint'] = ordinal_encoder.fit_transform(df[['maint']])
df['doors'] = ordinal_encoder.fit_transform(df[['doors']])
df['persons'] = ordinal_encoder.fit_transform(df[['persons']])
df['lug_boot'] = ordinal_encoder.fit_transform(df[['lug_boot']])
df['safety'] = ordinal_encoder.fit_transform(df[['safety']])
df['class'] = ordinal_encoder.fit_transform(df[['class']])

#Modelo KNeighborsClassifier
## Dado el desbalance de las etiquetas objetivo, aplicamos stratify=y en la partición de lo datos y aseguramos proporciones iguales en los set de entrenamiento y testeo

In [157]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score


# Dividir los datos en características (X) y objetivos (y)
X = df.drop(columns=['class'])  # Características
y = df[['class']]  # Etiquetas

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Inicializar los clasificadores de vecinos cercanos para cada columna objetivo
knn = KNeighborsClassifier(n_neighbors=5)

# # Entrenar el modelo de vecinos cercanos
knn.fit(X_train, y_train)

# # Predecir las etiquetas para los datos de prueba
y_pred = knn.predict(X_test)


/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


In [159]:
# # Calcular la precisión del modelo
precision = accuracy_score(y_test, y_pred)

print("Precisión del modelo de vecinos cercanos:", precision)

Precisión del modelo de vecinos cercanos: 0.9104046242774566


#Tecnica de escalado estandar en modelo para mejorar rendimiento

In [161]:
#Escalamos Modelo
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Ajustamos modelo
knn.fit(X_train_scaled, y_train)
# score
print("Knn test accuracy: {:.2f}".format(knn.score(X_test_scaled, y_test)))

Knn test accuracy: 0.96


/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


#Resumen de clasifición

In [162]:
from sklearn.metrics import classification_report

# Realizar predicciones en los datos de prueba
y_pred = knn.predict(X_test_scaled)

# Generar el informe de clasificación
reporte_clasificacion = classification_report(y_test, y_pred)

# Imprimir el informe de clasificación
print(reporte_clasificacion)


              precision    recall  f1-score   support

         0.0       0.87      0.97      0.92        77
         1.0       0.86      0.43      0.57        14
         2.0       0.99      1.00      0.99       242
         3.0       1.00      0.77      0.87        13

    accuracy                           0.96       346
   macro avg       0.93      0.79      0.84       346
weighted avg       0.96      0.96      0.96       346



#Conclusiones

El modelo alcanzo un accuracy del 0.96 con alta precisión para evaluar los vehiculos y clasificarlos correctamente en sus categorias, de estas obtenemos valores sobresalientes para las etiquetas 2 y 3,con una precision del 0.99 y 1.0 respectivamnete.

* 'acc' codificado como 0
* 'good' codificado  como 1
* 'unacc' codificado  como 2
* 'vgood' codificado  como 3


#Modelo Regresión Logistica

In [202]:
# Cargar los datos
url ='https://archive.ics.uci.edu/static/public/19/data.csv'
df = pd.read_csv(url, delimiter=',')



In [203]:
# Aplicar codificación one-hot a las variables categóricas

df_encoded = pd.get_dummies(df, drop_first=True)


In [206]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Dividir los datos en características (X) y variable objetivo (y)
X = df.drop(columns=['class'])
y = df['class']  # Variable objetivo: class

# Aplicar codificación one-hot a las variables categóricas excepto 'class'
X_encoded = pd.get_dummies(X, drop_first=True)

# Dividir los datos en conjuntos de entrenamiento y prueba (80% entrenamiento, 20% prueba)
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2,random_state=42)

# Construir el modelo de regresión logística
model = LogisticRegression(max_iter=1000)

# Entrenar el modelo con los datos de entrenamiento
model.fit(X_train, y_train)

# Predecir sobre el conjunto de prueba
y_pred = model.predict(X_test)

# Evaluar el rendimiento del modelo
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.8988439306358381


In [207]:
#Escalamos Modelo
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Ajustamos modelo
model.fit(X_train_scaled, y_train)
# score
print("LogisticRegression test accuracy: {:.2f}".format(model.score(X_test_scaled, y_test)))

LogisticRegression test accuracy: 0.93


In [201]:
from sklearn.linear_model import LogisticRegression

# Crear el modelo de regresión logística con regularización L2 (Ridge)
model = LogisticRegression(penalty='l2', C=1.0)

# Entrenar el modelo
model.fit(X_train, y_train)

# Predecir las etiquetas para los datos de prueba
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8988439306358381


In [196]:
from sklearn.metrics import classification_report

# Realizar predicciones en los datos de prueba
y_pred = model.predict(X_test_scaled)

# Generar el informe de clasificación
reporte_clasificacion_logistica = classification_report(y_test, y_pred)

# Imprimir el informe de clasificación
print(reporte_clasificacion_logistica)

              precision    recall  f1-score   support

         acc       0.88      0.82      0.85        83
        good       0.60      0.82      0.69        11
       unacc       0.96      0.97      0.97       235
       vgood       0.94      0.94      0.94        17

    accuracy                           0.93       346
   macro avg       0.85      0.89      0.86       346
weighted avg       0.93      0.93      0.93       346



#Conclusiones

El modelo alcanzo un accuracy del 0.93 con alta precisión para evaluar los vehiculos y clasificarlos correctamente en sus categorias, de estas obtenemos valores sobresalientes para las etiquetas unacc y vgood,con una precision del 0.96 y 0.94 respectivamnete.


# Modelo Random Forest

In [185]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder

In [186]:
#Nombres de las columnas
column_names = ["buying", "maint", "doors", "persons", "lug_boot", "safety", "class"]

In [187]:
url ='https://raw.githubusercontent.com/Fire2025/Visualizacion_CienciadeDatos/main/car.data.csv'
df = pd.read_csv(url, delimiter = ',', header=None, names=column_names)

In [188]:
from sklearn.preprocessing import LabelEncoder

label_encoders = {}
for column in df.columns:
    le = LabelEncoder()
    # Original unique categories
    print(f'Original unique categories in {column}: {df[column].unique()}')

    df[column] = le.fit_transform(df[column])

    # Transformed unique categories
    print(f'Transformed unique categories in {column}: {df[column].unique()}')

    label_encoders[column] = le
    #imprimir el LabelEncoder para ver cómo se ha configurado internamente
    print(f'Label encoder for {column}: {le.classes_}\n')


Original unique categories in buying: ['vhigh' 'high' 'med' 'low']
Transformed unique categories in buying: [3 0 2 1]
Label encoder for buying: ['high' 'low' 'med' 'vhigh']

Original unique categories in maint: ['vhigh' 'high' 'med' 'low']
Transformed unique categories in maint: [3 0 2 1]
Label encoder for maint: ['high' 'low' 'med' 'vhigh']

Original unique categories in doors: ['2' '3' '4' '5more']
Transformed unique categories in doors: [0 1 2 3]
Label encoder for doors: ['2' '3' '4' '5more']

Original unique categories in persons: ['2' '4' 'more']
Transformed unique categories in persons: [0 1 2]
Label encoder for persons: ['2' '4' 'more']

Original unique categories in lug_boot: ['small' 'med' 'big']
Transformed unique categories in lug_boot: [2 1 0]
Label encoder for lug_boot: ['big' 'med' 'small']

Original unique categories in safety: ['low' 'med' 'high']
Transformed unique categories in safety: [1 2 0]
Label encoder for safety: ['high' 'low' 'med']

Original unique categories 

In [189]:
# Dividir los datos en conjuntos de entrenamiento y prueba
# Se elimina la variable objetivo de X
X = df.drop('class', axis=1)
y = df['class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [190]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

rf_model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')

# Entrenar el modelo
rf_model.fit(X_train, y_train)

# Predicciones
y_pred = rf_model.predict(X_test)

# Reporte de clasificación
report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.96      0.88      0.92       103
           1       0.59      0.87      0.70        15
           2       0.99      1.00      0.99       294
           3       0.95      0.95      0.95        20

    accuracy                           0.96       432
   macro avg       0.87      0.92      0.89       432
weighted avg       0.97      0.96      0.96       432



# Resultados obtenidos con el modelo de Random Forest:

El modelo de Random Forest ha demostrado un desempeño sobresaliente en este conjunto de datos. Su capacidad para manejar efectivamente el desequilibrio de clases y sus características intrínsecas lo hacen ideal para este tipo de problemas de clasificación.

 **Análisis del Reporte de Clasificación:**
  
1.   Precisión (Precision): Es muy alta para todas las clases, especialmente para las clases 0, 2 y 3, donde es 0.96, 0.99 y 0.95, respectivamente. Esto indica que cuando el modelo predice una clase, es muy probable que la predicción sea correcta.
2.   Recall (Recuperación): También es excepcionalmente alto para casi todas las clases. Destaca el recall de la clase 2 con un perfecto 1.00, indicando que el modelo identificó correctamente todos los casos reales de esta clase en el conjunto de prueba. Las clases 1 y 3 también muestran valores altos de recall, 0.87 y 0.95 respectivamente.
3.   F1-Score: Este balance entre precisión y recall es útil cuando las clases están desbalanceadas como en este caso. Los valores de F1-score son excelentes, destacando la clase 2 con 0.99. Esto refleja un equilibrio muy bueno entre precisión y recall.
4.   Soporte (Support): Muestra el número de ocurrencias de cada clase en el conjunto de prueba, ayudando a entender el contexto de las métricas. Aunque la clase 1 tiene un soporte menor (15), el modelo aún maneja bien su predicción.
5.   Accuracy (Exactitud): La exactitud global del modelo es de 0.96, lo que indica que el 96% de todas las predicciones fueron correctas.
